In [1]:
import pandas as pd
import numpy as np
import geatpy as ea
from multiprocessing import Pool as ProcessPool
import multiprocessing as mp
from multiprocessing.dummy import Pool as ThreadPool

In [2]:
场地处理能力df = pd.read_excel('../../preprocess/场地处理能力.xlsx')
场地处理能力df.set_index('场地', inplace=True)
线路运输能力df = pd.read_excel('../../preprocess/线路运输能力.xlsx')
rawData = pd.read_excel('../../T1/解题/时间窗口为5-第一问结果LTSM-物流网络历史货量数据.xlsx')
rawDataMelt = pd.melt(pd.merge(rawData.iloc[:, :2], rawData.iloc[:, -31:], left_index=True, right_index=True),
                      id_vars=['场地1', '场地2'], value_name='货量', var_name='日期')
totalEdgeLen = len(rawData)
print(f'总线路数量：{totalEdgeLen}')
print(rawDataMelt.dtypes)
rawDataMelt

总线路数量：1049
场地1     int64
场地2     int64
日期     object
货量      int64
dtype: object


,场地1,场地2,日期,货量
0,1,8,2023-01-01,3
1,2,3,2023-01-01,1
2,2,4,2023-01-01,2
3,2,5,2023-01-01,10
4,2,8,2023-01-01,2
...,...,...,...,...
32514,76,8,2023-01-31,15
32515,76,10,2023-01-31,13
32516,76,14,2023-01-31,1
32517,76,62,2023-01-31,4


In [3]:
线路运输能力df = 线路运输能力df.set_index(['场地1', '场地2'])
线路运输能力df


货量
场地1 场地2      
1   8    2190
2   3    4756
    4    7252
    5    3797
    8    1604
...       ...
76  8    2920
    10   2455
    14   1175
    62   1429
77  6    1460

[1049 rows x 1 columns]

In [4]:
# 与DC5相关的线路
向dc5发货的场地 = rawData[(rawData['场地2'] == 5)]['场地1']
print(向dc5发货的场地)
print(f'发送货物到DC5的路线数量：{len(向dc5发货的场地)}')
可用的分发线路 = rawData[rawData['场地1'].isin(向dc5发货的场地)]
可用的分发线路

3        2
10       3
27       4
79       7
93       8
        ..
999     71
1011    72
1020    73
1035    74
1043    76
Name: 场地1, Length: 67, dtype: int64
发送货物到DC5的路线数量：67


,场地1,场地2,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,...,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30,2023-01-31
1,2,3,9,9,9,9,9,9,9,9,...,3,4,4,4,4,4,4,4,4,4
2,2,4,2,2,2,2,2,2,2,2,...,4,4,5,5,5,5,5,5,5,5
3,2,5,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,5
4,2,8,2,2,2,2,2,2,2,2,...,4,4,4,4,4,4,4,4,4,4
5,2,9,33,33,33,33,33,33,33,33,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,76,5,3,3,3,3,3,3,3,3,...,4,5,5,5,6,6,6,7,7,8
1044,76,8,4,4,4,4,4,4,4,4,...,11,11,12,12,13,13,14,14,15,15
1045,76,10,3,3,3,3,3,3,3,3,...,9,10,10,11,11,12,12,13,13,13
1046,76,14,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1


In [7]:
可用的分发线路tuple = list(map(tuple, 可用的分发线路.iloc[:, :2].values.tolist()))
len(可用的分发线路tuple)

998

In [6]:
可用的分发线路2023年 = pd.merge(可用的分发线路.iloc[:, :2], 可用的分发线路.iloc[:, -31:], left_index=True,
                                right_index=True)
可用的分发线路2023年

,场地1,场地2,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,...,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30,2023-01-31
1,2,3,1,1,1,1,1,1,1,2,...,3,4,4,4,4,4,4,4,4,4
2,2,4,2,2,2,3,3,3,3,3,...,4,4,5,5,5,5,5,5,5,5
3,2,5,10,10,10,9,9,9,9,8,...,6,6,6,6,6,6,6,6,6,5
4,2,8,2,2,2,2,2,3,3,3,...,4,4,4,4,4,4,4,4,4,4
5,2,9,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,76,5,2,2,2,2,2,2,2,3,...,4,5,5,5,6,6,6,7,7,8
1044,76,8,4,4,4,5,5,5,5,5,...,11,11,12,12,13,13,14,14,15,15
1045,76,10,4,4,4,4,5,5,5,5,...,9,10,10,11,11,12,12,13,13,13
1046,76,14,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [10]:
可用的分发线路2023年[(可用的分发线路2023年['场地1'] == 69) & (可用的分发线路2023年['场地2'] == 8)]

,场地1,场地2,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,...,2023-01-22,2023-01-23,2023-01-24,2023-01-25,2023-01-26,2023-01-27,2023-01-28,2023-01-29,2023-01-30,2023-01-31
981,69,8,10063,10627,11117,13003,12058,12030,12320,12612,...,13153,13161,13153,13145,13137,13126,13121,13113,13104,13095


In [19]:
线路运输能力df

货量
场地1 场地2      
1   8    2190
2   3    4756
    4    7252
    5    3797
    8    1604
...       ...
76  8    2920
    10   2455
    14   1175
    62   1429
77  6    1460

[1049 rows x 1 columns]

In [50]:
全年线路负载率df = pd.melt(rawData, id_vars=['场地1', '场地2'], value_name='货量', var_name='日期').groupby(
    by=['场地1', '场地2'])[['货量']].mean()

# 全年线路负载率df.sort_values(['货量'], ascending=False)
全年线路负载率df

货量
场地1 场地2          
1   8    3.040736
2   3    6.353482
    4    9.684625
    5    5.281209
    8    2.241787
...           ...
76  8    4.187911
    10   3.540079
    14   1.584757
    62   1.977661
77  6    2.067017

[1049 rows x 1 columns]

In [51]:
全年线路负载率df['负载率'] = 全年线路负载率df.values / 线路运输能力df[
    线路运输能力df.index.isin(全年线路负载率df.index)].values
全年线路负载率df

货量       负载率
场地1 场地2                    
1   8    3.040736  0.001388
2   3    6.353482  0.001336
    4    9.684625  0.001335
    5    5.281209  0.001391
    8    2.241787  0.001398
...           ...       ...
76  8    4.187911  0.001434
    10   3.540079  0.001442
    14   1.584757  0.001349
    62   1.977661  0.001384
77  6    2.067017  0.001416

[1049 rows x 2 columns]

In [52]:
全年线路负载率df.reset_index().to_excel('全年线路负载率df.xlsx', index=False)